this notebook is used to calculate evolution of densities (central/column density) and Mach number

In [ ]:
from postprocessing import *
from scipy.spatial import KDTree
from scipy.integrate import trapz

In [ ]:
mpl.rcParams['figure.dpi']= 200

## density in the center

In [ ]:
simulation_directory = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100_m12/turb')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
central_density_max = []
central_density_mean = []

In [ ]:
i_file = -1  # skip snap 0
while True:
    i_file += 1
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data) * unit_time_in_megayr
    x, y, z = snap_data['PartType0/Coordinates'][:].T
    mask = ((np.sqrt((x - 450) ** 2 + (y - 500) ** 2 + (z - 500) ** 2) < 50))
    n_max = np.max(snap_data['PartType0/Density'][mask]) * rho_to_numdensity
    n_mean = np.mean(snap_data['PartType0/Density'][mask]) * rho_to_numdensity 
    central_density_max.append([time, n_max])
    central_density_mean.append([time, n_mean])

In [ ]:
central_density_max = np.array(central_density_max).T
central_density_mean = np.array(central_density_mean).T

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(central_density_mean[0], central_density_mean[1])
# ax.scatter(central_density_mean[0][12], central_density_mean[1][12],c='red', zorder=100)
# ax.scatter(central_density_mean[0][20], central_density_mean[1][20],c='red', zorder=100)
# ax.scatter(central_density_mean[0][28], central_density_mean[1][28],c='red', zorder=100)
ax.set_xlabel('t [Myr]', fontsize=12)
ax.set_ylabel('mean density inside lobes [cm$^{-3}$]', fontsize=10)
plt.show()

## column density

In [ ]:
simulation_directory = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d100_m12/turb')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
center = 500
length = 200
x_grid_right = np.linspace(center, center + length, 501)
x_grid_left = np.linspace(center - length, center, 501)
y_grid = np.zeros_like(x_grid_right)
z_grid = np.zeros_like(x_grid_right)
column_density_right = []
column_density_left = []

In [ ]:
i_file = -1  # skip snap 0
while True:
    i_file += 1
    print(i_file)
    filename = "snap_%03d.hdf5" % (i_file)
    try:
        snap_data = h5py.File(output_directory + filename, "r")
    except:
        break
    time = get_time_from_snap(snap_data) * unit_time_in_megayr
    tree = KDTree(snap_data['PartType0/Coordinates'][:])
    
    r, indeces = tree.query(np.array([x_grid_left, y_grid, z_grid]).T)
    densities = []
    for i in indeces:
        densities.append(snap_data['PartType0/Density'][i] * rho_to_numdensity)
    column_density_left.append([time, trapz(y=densities, x=x_grid_left)])
    
    r, indeces = tree.query(np.array([x_grid_right, y_grid, z_grid]).T)
    densities = []
    for i in indeces:
        densities.append(snap_data['PartType0/Density'][i] * rho_to_numdensity)
    column_density_right.append([time, trapz(y=densities, x=x_grid_right)])

In [ ]:
column_density_left = np.array(column_density_left).T
column_density_right = np.array(column_density_right).T

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(column_density_left[0], column_density_left[1], label='to the left')
ax.plot(column_density_right[0], column_density_right[1], label='to the right')
# ax.axvline(column_density_left[0][12],c='tab:red', zorder=100)
# ax.axvline(column_density_left[0][20],c='tab:red', zorder=100)
# ax.axvline(column_density_left[0][28],c='tab:red', zorder=100)
ax.set_xlabel('t [Myr]', fontsize=12)
ax.set_ylabel('column density [cm$^{-2}$]', fontsize=10)
plt.legend()
plt.show()

## Mach number

In [ ]:
simulation_directory = str('/n/holystore01/LABS/hernquist_lab/Users/borodina/turb_drive_center_d10_m8/turb_2')
output_directory = simulation_directory+"/output/"
figures_directory = simulation_directory + "/output/figures/"

In [ ]:
turb = pd.read_csv(simulation_directory + '/output/turb.txt', sep=' ', names=['t',
                                                           'mach',
                                                           'etot/mass', 
                                                           'dudt/mass_drive', 
                                                           'dudt/mass_diss', 
                                                           'energyinjected/mass', 
                                                           'energydissipated/mass'])

In [ ]:
n_early = abs(turb['t'] - central_density_mean[0][4]).argmin()
n_mid = abs(turb['t'] - central_density_mean[0][5]).argmin()
n_last = abs(turb['t'] - central_density_mean[0][12]).argmin()

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.scatter(turb['t'] * unit_time_in_megayr, turb['mach'], s=0.01)
ax.scatter(turb['t'][n_early] * unit_time_in_megayr, turb['mach'][n_early],c='red', zorder=100)
ax.scatter(turb['t'][n_mid] * unit_time_in_megayr, turb['mach'][n_mid],c='red', zorder=100)
ax.scatter(turb['t'][n_last] * unit_time_in_megayr, turb['mach'][n_last],c='red', zorder=100)
ax.set_xlabel('t [Myr]', fontsize=15)
ax.set_ylabel('M', fontsize=15)
plt.show()